<a href="https://colab.research.google.com/github/crapptrapp/sadGPT/blob/main/sadGPT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Gradio Demo: text_generation
### This text generation demo takes in input text and returns generated text. It uses the Transformers library to set up the model and has two examples.
        

In [1]:
!pip install -q gradio git+https://github.com/huggingface/transformers gradio torch

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.8/15.8 MB 66.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.5/84.5 KB 10.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.7/45.7 KB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.2/199.2 KB 24.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 36.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.5/71.5 KB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.9/140.9 KB 16.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.4/56.4 KB 6.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.5/50.5 KB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.8/57.8 KB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━

In [10]:
from transformers import pipeline

generator = pipeline('text-generation', model='gpt2')

def generate(text):
    result = generator(text, max_length=300, num_return_sequences=1)
    #result = generator(text, max_length=30, num_return_sequences=1)
    return result[0]["generated_text"]

examples = [
    ["The Moon's orbit around Earth has"],
    ["The smooth Borealis basin in the Northern Hemisphere covers 40%"],
]



In [6]:
generate("Empowered by life energy, imagination shall manifest as Reality.")

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


'Empowered by life energy, imagination shall manifest as Reality. (2) The essence of life shall be like an ocean rising up and passing through'

In [11]:
generate("If we want humanity to enjoy the fruits of science and technology rather than use them to destroy ourselves and the entire planet, the most important thing right now is to raise Human Consciousness.")

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


'If we want humanity to enjoy the fruits of science and technology rather than use them to destroy ourselves and the entire planet, the most important thing right now is to raise Human Consciousness.\n\nAdvertisements'

In [ ]:
import gradio as gr

demo = gr.Interface(
    fn=generate,
    inputs=gr.inputs.Textbox(lines=5, label="Input Text"),
    outputs=gr.outputs.Textbox(label="Generated Text"),
    examples=examples
)

demo.launch()